In [1]:
import time
import pandas as pd
from datetime import datetime

import bernielib as bl

bl.listSerialPorts()

['COM3', 'COM4']

In [2]:
ber = bl.robot(cartesian_port_name='COM3', loadcell_port_name='COM4')

In [11]:
ber.home()

# Preparing diluted stock solution

Diluting 10 times

In [4]:
dna_stock = bl.createSample('eppendorf', 'dna_stock', ber.samples_rack, 0, 9, 200)
dna_stock_2 = bl.createSample('eppendorf', 'dna_stock_2', ber.samples_rack, 0, 8, 0)
water = bl.createSample('25ml', 'water', ber.reagents_rack, 0, 0, 25000)

In [5]:
script = pd.read_csv('mixing_pattern_eppendorf.csv')
ber.pickUpNextTip()
ber.move(z=50)
ber.transferLiquid(water, dna_stock_2, 270)
ber.transferLiquid(dna_stock, dna_stock_2, 30, v_immerse_dispense=1600)
ber.move(z=50)
ber.mixByScript(dna_stock_2, script)
ber.pipetteHome()
ber.move(z=50)
ber.dumpTipToWaste()

# Protocol functions

In [22]:
def calcBeadsVolumeToAdd(robot, samples_list, cutoff_list):
    v_beads_list = []
    for sample, dna_size in zip(samples_list, cutoff_list):
        v_beads = robot.calcBeadsVol(sample, dna_size)
        v_beads_list.append(v_beads)
    return v_beads_list

def waitAfterTimestamp(timestamp, delay):
    new_ts = time.time()
    while (new_ts - timestamp) < delay:
        time.sleep(1)
        new_ts = time.time()

def waitAndMixByScript(robot, sample, timestamp, delay, mix_script, tip_col, tip_row):
    new_ts = time.time()
    robot.pickUpTip(tip_col, tip_row)
    robot.move(z=50)
    while (new_ts - timestamp) < delay:
        robot.mixByScript(sample, mix_script)
        robot.move(z=50)
        new_ts = time.time()
    robot.dumpTipToPosition(tip_col, tip_row)

def addBeads(robot, sample, beads, v_beads, sample_mix_scenario, beads_mix_scenario, z_safe=50):
    robot.move(z=z_safe)
    #robot.pickUpNextTip()
    #robot.move(z=z_safe)
    # Mixing beads before experiment
    #robot.mixByScript(beads, beads_mix_scenario)
    # Transferring beads to sample
    robot.transferLiquid(beads, sample, v_beads)
    # Mixing sample with beads
    robot.mixByScript(sample, sample_mix_scenario)
    robot.move(z=z_safe)
    #robot.dumpTipToWaste()
    robot.move(z=z_safe)

def addBeadsToAll(robot, samples_list, v_beads_list, beads, sample_mix_scenario, beads_mix_scenario):
    robot.moveMagnetsAway(poweroff=True)
    robot.pickUpNextTip()
    robot.move(z=50)
    robot.mixByScript(beads, beads_mix_scenario)
    robot.move(z=50)
    
    for sample, v_beads in zip(samples_list, v_beads_list):
        robot.transferLiquid(beads, sample, v_beads, touch_wall=False)
    
    counter = 0
    for sample, v_beads in zip(samples_list, v_beads_list):
        if counter != 0:
            robot.move(z=50)
            robot.pickUpNextTip()
        
        robot.move(z=50)
        
        robot.mixByScript(sample, sample_mix_scenario)
            
        if counter == 0:
            timestamp = time.time()
        counter += 1
        
        robot.move(z=50)
        #robot.dumpTipToWaste()
        robot.returnTipBack()
    
    return timestamp

def removeSupernatant(robot, sample, waste, z_safe=50, delay=0.5):
    robot.pickUpNextTip()
    robot.move(z=z_safe)
    z0 = robot._getTubeZBottom(sample)
    while sample.getVolume() > 200:
        robot.movePipetteToVolume(200)
        robot.moveToSample(sample)
        robot.move(z=z0-0.5)
        robot.movePipetteToVolume(0)
        new_vol = sample.getVolume() - 200
        if new_vol < 0:
            new_vol = 0
        sample.setVolume(new_vol)
        robot.move(z=z_safe)
        robot.dispenseLiquid(waste, 200, blow_extra=True)
        robot.move(z=z_safe)
    robot.moveToSample(sample)
    robot.movePipetteToVolume(250)
    robot.move(z=z0)
    robot.movePipetteToVolume(50)
    time.sleep(delay)
    robot.move(z=z0-0.5)
    time.sleep(delay)
    robot.moveAxisDelta('X', 1.5)
    robot.movePipetteToVolume(40)
    time.sleep(delay/4.0)
    robot.moveAxisDelta('X', -1.5)
    robot.moveAxisDelta('Y', 1.5)
    robot.movePipetteToVolume(30)
    time.sleep(delay/4.0)
    robot.moveAxisDelta('Y', -1.5)
    robot.moveAxisDelta('X', -1.5)    
    robot.movePipetteToVolume(20)
    time.sleep(delay/4.0)
    robot.moveAxisDelta('X', 1.5)
    robot.moveAxisDelta('Y', -1.5)
    robot.movePipetteToVolume(0)
    time.sleep(delay/4.0)
    robot.move(z=z_safe)
    robot.dispenseLiquid(waste, 250, blow_extra=True)
    robot.move(z=z_safe)
    robot.dumpTipToWaste()

def removeSupernatantFast(robot, sample, waste, z_safe=50, delay=0.5):
    robot.pickUpNextTip()
    robot.move(z=z_safe)
    z0 = robot._getTubeZBottom(sample)
    while sample.getVolume() > 0:
        robot.movePipetteToVolume(200)
        robot.moveToSample(sample)
        robot.move(z=z0-0.5)
        robot.movePipetteToVolume(0)
        new_vol = sample.getVolume() - 200
        if new_vol < 0:
            new_vol = 0
        sample.setVolume(new_vol)
        time.sleep(delay)
        robot.move(z=z_safe)
        robot.dispenseLiquid(waste, 200, blow_extra=True)
        robot.move(z=z_safe)
    robot.dumpTipToWaste()

def removeSupernatantAllSamples(robot, samples_list, waste, how='fast'):
    counter = 0
    for sample in samples_list:
        if how == 'fast':
            removeSupernatantFast(robot, sample, waste)
        else:
            removeSupernatant(robot, sample, waste)
        if counter == 0:
            sample_dried_timestamp = time.time()
        counter += 1
    return sample_dried_timestamp

def add80PctEthanol(robot, samples_list, ethanol, volume, z_safe=50):
    robot.pickUpNextTip()
    robot.move(z=z_safe)
    
    counter = 0
    for sample in samples_list:
        robot.transferLiquid(ethanol, sample, volume, touch_wall=False)
        if counter == 0:
            ethanol_added_time = time.time()
        counter += 1
    
    robot.move(z=z_safe)
    robot.dumpTipToWaste()
    
    return ethanol_added_time

def elutionMix(robot, sample, volume, delay=0.5):
    z0 = robot._getTubeZBottom(sample)
    z_top = sample.getSampleTopAbsZ(added_length=robot._calcExtraLength())
    robot.movePipetteToVolume(0)
    robot.movePipetteToVolume(volume+5)
    robot.movePipetteToVolume(volume)
    robot.move(z=z0-0.5)
    robot.movePipetteToVolume(0)
    time.sleep(delay)
    # Washing steps, moving along the wall
    # 1
    z_curr = z_top + 24
    #z_curr = sample.calcAbsLiquidLevelFromVol(500, added_length=robot._calcExtraLength())
    robot.move(z=z_curr)
    robot.moveAxisDelta('X', 3.0)
    robot.movePipetteToVolume(volume/4.0)
    time.sleep(delay/4.0)
    # 2
    z_curr = z_top + 28
    #z_curr = sample.calcAbsLiquidLevelFromVol(300, added_length=robot._calcExtraLength())
    robot.move(z=z_curr)
    #robot.moveAxisDelta('X', -0.629)
    robot.movePipetteToVolume(2 * (volume/4.0))
    time.sleep(delay/4.0)
    # 3
    z_curr = z_top + 32
    #z_curr = sample.calcAbsLiquidLevelFromVol(150, added_length=robot._calcExtraLength())
    robot.move(z=z_curr)
    robot.moveAxisDelta('X', -0.629)
    robot.movePipetteToVolume(3 * (volume/4.0))
    time.sleep(delay/4.0)
    # 4
    z_curr = z_top + 36
    #z_curr = sample.calcAbsLiquidLevelFromVol(150, added_length=robot._calcExtraLength())
    robot.move(z=z_curr)
    robot.moveAxisDelta('X', -0.629)
    robot.movePipetteToVolume(volume)
    time.sleep(delay/4.0)
    
    x, y = sample.getCenterXY()
    robot.move(x=x, y=y)
    z_curr = sample.calcAbsLiquidLevelFromVol(volume+100, added_length=robot._calcExtraLength())
    robot.move(z=z_curr)
    
    robot.movePipetteToVolume(volume+50)
    time.sleep(delay)
    robot.movePipetteToVolume(0)

def elutionMixLrgVol(robot, sample, volume, delay=0.5):
    z0 = robot._getTubeZBottom(sample)
    z_top = sample.getSampleTopAbsZ(added_length=robot._calcExtraLength())
    
    # Uptaking
    robot.movePipetteToVolume(0)
    robot.movePipetteToVolume(volume+5)
    robot.movePipetteToVolume(volume)
    robot.move(z=z0-0.8)
    robot.movePipetteToVolume(0)
    time.sleep(delay)
    
    # Ejecting liquid
    z_curr = sample.calcAbsLiquidLevelFromVol(1000, added_length=robot._calcExtraLength())
    robot.move(z=z_curr)
    robot.moveAxisDelta('X', 3.4)
    robot.movePipetteToVolume(volume+50)
    z_curr = sample.calcAbsLiquidLevelFromVol(500, added_length=robot._calcExtraLength())
    robot.move(z=z_curr)
    
    # To back position
    x, y = sample.getCenterXY()
    robot.move(x=x, y=y)
    
    robot.movePipetteToVolume(0)

#TODO: make safe_z a general property of the robot
def elute(robot, sample, eluent, volume, mix_delay=0.5, mix_times=5, safe_z=50):
    robot.moveMagnetsAway(poweroff=True)
    robot.pickUpNextTip()
    robot.move(z=safe_z)
    robot.transferLiquid(eluent, sample, volume)
    for i in range(mix_times):
        elutionMixLrgVol(robot, sample, volume)
        elutionMixLrgVol(robot, sample, volume)
        elutionMix(robot, sample, volume)
    elution_start_time = time.time()
    elutionMix(robot, sample, volume)
    elutionMix(robot, sample, volume)
    
    robot.move(z=safe_z)
    robot.returnTipBack()
    #robot.dumpTipToWaste()
    sample.setVolume(volume)
    return elution_start_time

def eluteAllSamples(robot, samples_list, eluent, V_eluent, mix_delay=0.5, mix_times=4, safe_z=50):
    counter = 0
    for sample in samples_list:
        ts = elute(ber, sample, eluent, V_eluent, mix_delay=mix_delay, mix_times=mix_times, safe_z=safe_z)
        print ()
        if counter == 0:
            elution_start_timestamp = ts
        counter += 1
    return elution_start_timestamp

def separateEluate(robot, eluate_tube, result_tube, pipette_delay=1.0, z_above_bottom=0.2, safe_z=50):
    robot.pickUpNextTip()       # Getting a new tip
    robot.move(z=safe_z)        # Moving up so the tip does not hit anything
    # Uptaking liquid
    v = eluate_tube.getVolume()      # Volume in the tube with the mix of the eluted DNA and beads
    v = v * 2                        # Will uptake extra volume
    robot.moveToSample(eluate_tube)  # Moving XY to the sample with the eluted DNA and the beads.
    robot.movePipetteToVolume(v+5)   # 
    robot.movePipetteToVolume(v)     # Plunger down to the volume of the DNA solution
    z0 = robot._getTubeZBottom(eluate_tube)  # Bottom of the tube with eluted DNA and beads
    robot.move(z=z0-z_above_bottom) # Moving to the bottom of the tube with eluted DNA and beads
    robot.movePipetteToVolume(v/2.0)    # Plunger half-way up
    time.sleep(pipette_delay)       # Waiting
    robot.move(z=z0-z_above_bottom-1.0, speed_z=30)
    robot.move(z=z0-z_above_bottom, speed_z=30)
    robot.movePipetteToVolume(0.0)  # Plunger all the way up
    time.sleep(pipette_delay)       # Waiting
    robot.move(z=z0-z_above_bottom-1.0, speed_z=30)
    time.sleep(pipette_delay)       # Waiting
    # Dispensing liquid
    robot.moveToSample(result_tube)  # Moving XY to the empty tube for the purified DNA
    robot.movePipetteToVolume(v+5)   # Plunger down to the volume of the DNA solution that is currently in the tip
    time.sleep(pipette_delay)        # Waiting for water to drip
    robot.movePipetteToVolume(200)   # Moving plunger all the way down (removing any extra drops from the tip)
    time.sleep(pipette_delay)        # Waiting for the water to drip
    robot.touchWall(result_tube)     # Touching the wall to remove a potential drop of DNA solution from the tip end.
    robot.movePipetteToVolume(0)    # Plunger back up
    robot.move(z=safe_z)            # Moving Z up so the tip does not hit anything
    ber.dumpTipToWaste()            # Discarding the tip

def oneStagePurification(robot, N_samples, initial_vol_list, cutoff_list, V_avail_beads, V_avail_water, V_avail_ethanol, 
                         beads_rack, beads_col, beads_row, sample_mix_scenario, beads_mix_scenario,
                         V_waste=0, T_absorb=600, T_pull=60, T_wash=30, T_dry=300, T_elute=600,
                         V_wash=200, V_elute=30):
    
    print("Experiment started ", datetime.now().strftime("%H:%M:%S"))
    samples_list = bl.createSamplesToPurifyList(robot, initial_vol_list)
    result_list = bl.createPurifiedSamplesList(robot, N_samples)
    beads = bl.createSample('eppendorf', 'beads', beads_rack, beads_col, beads_row, V_avail_beads)
    waste = bl.createSample('25ml', 'liquid_waste', robot.reagents_rack, 0, 1, V_waste)
    water = bl.createSample('25ml', 'water', robot.reagents_rack, 0, 0, V_avail_water)
    EtOH80pct = bl.createSample('5ml', 'EtOH80pct', robot.reagents_rack, 0, 4, V_avail_ethanol)
    
    v_beads_list = calcBeadsVolumeToAdd(robot, samples_list, cutoff_list=cutoff_list)
    # Adding beads
    timestamp_beads_added = addBeadsToAll(robot, samples_list, v_beads_list, beads, sample_mix_scenario, beads_mix_scenario)
    print("Beads added ", datetime.now().strftime("%H:%M:%S"))
    print("Waiting for DNA absorption")
    last_tip_col, last_tip_row = robot.last_tip_coord
    waitAndMixByScript(robot, samples_list[0], timestamp_beads_added, T_absorb, sample_mix_scenario, 
                       last_tip_col, last_tip_row)
    #waitAfterTimestamp(timestamp_beads_added, T_absorb)
    print("DNA absorption finished ", datetime.now().strftime("%H:%M:%S"))
    # Removing supernatant
    robot.moveMagnetsTowardsTube(poweroff=True)
    time.sleep(T_pull)
    print("Beads pulled to the side ", datetime.now().strftime("%H:%M:%S"))
    ts = removeSupernatantAllSamples(robot, samples_list, waste, how='fast')
    print("Supernatant removed ", datetime.now().strftime("%H:%M:%S"))
    # Ethanol wash
    # Wash 1
    timestamp_ethanol_added = add80PctEthanol(robot, samples_list, EtOH80pct, V_wash)
    print("Wash 1: ethanol added ", datetime.now().strftime("%H:%M:%S"))
    waitAfterTimestamp(timestamp_ethanol_added, T_wash)
    print("Wash 1: ethanol incubation finished ", datetime.now().strftime("%H:%M:%S"))
    ts = removeSupernatantAllSamples(robot, samples_list, waste, how='fast')
    print("Wash 1: ethanol removed ", datetime.now().strftime("%H:%M:%S"))
    # Wash 2
    timestamp_ethanol_added = add80PctEthanol(robot, samples_list, EtOH80pct, V_wash)
    print("Wash 2: ethanol added ", datetime.now().strftime("%H:%M:%S"))
    waitAfterTimestamp(timestamp_ethanol_added, T_wash)
    print("Wash 2: ethanol incubation finished ", datetime.now().strftime("%H:%M:%S"))
    timestamp_ethanol_removed = removeSupernatantAllSamples(robot, samples_list, waste, how='full')
    print("Wash 2: ethanol removed ", datetime.now().strftime("%H:%M:%S"))
    # Drying ethanol
    waitAfterTimestamp(timestamp_ethanol_removed, T_dry)
    print("Ethanol drying finished ", datetime.now().strftime("%H:%M:%S"))
    
    # Elution
    # Adding water
    elution_start_timestamp = eluteAllSamples(robot, samples_list, water, V_elute, mix_delay=0.5, mix_times=6, safe_z=50)
    print("Eluent added ", datetime.now().strftime("%H:%M:%S"))
    last_tip_col, last_tip_row = robot.last_tip_coord
    waitAndMixByScript(robot, samples_list[0], elution_start_timestamp, T_absorb, sample_mix_scenario, 
                       last_tip_col, last_tip_row)
    #waitAfterTimestamp(elution_start_timestamp, T_elute) # elution wait
    print("Elution incubation finished ", datetime.now().strftime("%H:%M:%S"))
    
    # Magnetic beads to the side of the tubes
    ber.moveMagnetsTowardsTube()
    time.sleep(T_pull)
    print("Beads pulled to the side ", datetime.now().strftime("%H:%M:%S"))
    # Moving liquid to the resulting tubes
    for sample, result in zip(samples_list, result_list):
        separateEluate(ber, sample, result)
    print("Eluate transferred to the new tube ", datetime.now().strftime("%H:%M:%S"))
    print("Experiment finished ", datetime.now().strftime("%H:%M:%S"))

In [7]:
script = pd.read_csv('mixing_pattern_eppendorf.csv')

# Custom ladder

## Protocol with 150 bp cutoff

In [8]:
dna_stock_2 = bl.createSample('eppendorf', 'dna_stock_2', ber.samples_rack, 0, 8, 300)
water = bl.createSample('50ml', 'water', ber.reagents_rack, 0, 0, 25000)

In [9]:
ber.pickUpNextTip()
sample = bl.createSamplesToPurifyList(ber, [0])[0]
ber.transferLiquid(dna_stock_2, sample, 30, v_immerse_dispense=1600, touch_wall=False)
ber.move(z=50)
ber.dumpTipToWaste()
ber.pipetteHome()

In [13]:
oneStagePurification(ber, N_samples=1, initial_vol_list=[30], 
                     cutoff_list=[150],
                     V_avail_beads=1000, V_avail_water=24000, V_avail_ethanol=5000, 
                     beads_rack=ber.samples_rack, beads_col=0, beads_row=11, 
                     sample_mix_scenario=script, beads_mix_scenario=script, 
                     T_absorb=200, T_dry=100, T_elute=200)

Experiment started  18:03:54
Beads added  18:05:35
Waiting for DNA absorption
DNA absorption finished  18:08:57
Beads pulled to the side  18:09:59
Supernatant removed  18:10:11
Wash 1: ethanol added  18:10:24
Wash 1: ethanol incubation finished  18:10:51
Wash 1: ethanol removed  18:11:04
Wash 2: ethanol added  18:11:17
Wash 2: ethanol incubation finished  18:11:44
Wash 2: ethanol removed  18:11:59
Ethanol drying finished  18:13:40

Eluent added  18:14:57
Elution incubation finished  18:18:20
Beads pulled to the side  18:19:21
Eluate transferred to the new tube  18:19:33
Experiment finished  18:19:33


In [12]:
ber.dumpTipToWaste()

In [20]:
ber.data

{'magnets_away_angle': 5.2,
 'magnets_near_tube_angle': 11.2,
 'stair_finding_step_list': [1, 0.2],
 'stair_finding_z_increment': 0.1,
 'stair_finding_z_retract_after_trigger': -1,
 'stair_finding_z_max_travel': 3,
 'stair_finding_z_load_threshold': 500,
 'z_max': 180,
 'x_max': 189,
 'y_max': 322,
 'added_tip_length': 41.6,
 'volume_to_position_slope': -0.15859245180518214,
 'volume_to_position_intercept': -0.958195131933648,
 'pipetting_delay': 0.2,
 'DNAsize_to_Vbeads': {'a': 0.499325349, 'b': -9.91043764, 'c': 25758.5836},
 'tip_drop_servo_up_angle': 2.5,
 'tip_drop_servo_down_angle': 7.3,
 'speed_XY': 50000,
 'speed_Z': 30000,
 'speed_pipette': 2500,
 'plunger_movement_when_dumping_tip': 35}

In [21]:
ber.move?

In [23]:
samples_list = bl.createSamplesToPurifyList(ber, [30])
result_list = bl.createPurifiedSamplesList(ber, 1)

In [24]:
separateEluate(ber, samples_list[0], result_list[0], pipette_delay=1.0, z_above_bottom=0.2)

## Protocol with 350 bp cutoff

In [6]:
dna_stock_2 = bl.createSample('eppendorf', 'dna_stock_2', ber.samples_rack, 0, 8, 270)
water = bl.createSample('50ml', 'water', ber.reagents_rack, 1, 1, 32000)

In [7]:
ber.pickUpNextTip()
sample = bl.createSamplesToPurifyList(ber, [0])[0]
ber.transferLiquid(dna_stock_2, sample, 30, v_immerse_dispense=1600, touch_wall=False)
ber.move(z=50)
ber.dumpTipToWaste()
ber.pipetteHome()

In [8]:
oneStagePurification(ber, N_samples=1, initial_vol_list=[30], 
                     cutoff_list=[350],
                     V_avail_beads=420, V_avail_water=30000, V_avail_ethanol=4500, 
                     beads_rack=ber.samples_rack, beads_col=0, beads_row=11, 
                     sample_mix_scenario=script, beads_mix_scenario=script, 
                     T_absorb=480, T_dry=300, T_elute=480)

Experiment started  14:20:41
Beads added  14:22:45
Waiting for DNA absorption
DNA absorption finished  14:31:04
Beads pulled to the side  14:32:06
Supernatant removed  14:32:35
Wash 1: ethanol added  14:33:08
Wash 1: ethanol incubation finished  14:33:30
Wash 1: ethanol removed  14:34:01
Wash 2: ethanol added  14:34:34
Wash 2: ethanol incubation finished  14:34:55
Wash 2: ethanol removed  14:35:31
Ethanol drying finished  14:40:31

Eluent added  14:43:21
Elution incubation finished  14:51:17
Beads pulled to the side  14:52:19
Eluate transferred to the new tube  14:52:47
Experiment finished  14:52:47


## Protocol with 550 bp cutoff

In [9]:
ber.pickUpNextTip()
sample = bl.createSamplesToPurifyList(ber, [0])[0]
ber.transferLiquid(dna_stock_2, sample, 30, v_immerse_dispense=1600, touch_wall=False)
ber.move(z=50)
ber.dumpTipToWaste()
ber.pipetteHome()

In [10]:
oneStagePurification(ber, N_samples=1, initial_vol_list=[30], 
                     cutoff_list=[550],
                     V_avail_beads=400, V_avail_water=30000, V_avail_ethanol=4000, 
                     beads_rack=ber.samples_rack, beads_col=0, beads_row=11, 
                     sample_mix_scenario=script, beads_mix_scenario=script, 
                     T_absorb=480, T_dry=300, T_elute=480)

Experiment started  14:59:46
Beads added  15:01:49
Waiting for DNA absorption
DNA absorption finished  15:09:57
Beads pulled to the side  15:10:59
Supernatant removed  15:11:28
Wash 1: ethanol added  15:12:01
Wash 1: ethanol incubation finished  15:12:22
Wash 1: ethanol removed  15:12:54
Wash 2: ethanol added  15:13:27
Wash 2: ethanol incubation finished  15:13:49
Wash 2: ethanol removed  15:14:24
Ethanol drying finished  15:19:24

Eluent added  15:22:15
Elution incubation finished  15:30:08
Beads pulled to the side  15:31:10
Eluate transferred to the new tube  15:31:38
Experiment finished  15:31:38


## Protocol with 1000 bp cutoff

In [11]:
ber.pickUpNextTip()
sample = bl.createSamplesToPurifyList(ber, [0])[0]
ber.transferLiquid(dna_stock_2, sample, 30, v_immerse_dispense=1600, touch_wall=False)
ber.move(z=50)
ber.dumpTipToWaste()
ber.pipetteHome()

In [12]:
oneStagePurification(ber, N_samples=1, initial_vol_list=[30], 
                     cutoff_list=[1000],
                     V_avail_beads=400, V_avail_water=30000, V_avail_ethanol=3500, 
                     beads_rack=ber.samples_rack, beads_col=0, beads_row=11, 
                     sample_mix_scenario=script, beads_mix_scenario=script, 
                     T_absorb=480, T_dry=300, T_elute=480)

Experiment started  15:37:16
Beads added  15:39:19
Waiting for DNA absorption
DNA absorption finished  15:47:26
Beads pulled to the side  15:48:27
Supernatant removed  15:48:56
Wash 1: ethanol added  15:49:30
Wash 1: ethanol incubation finished  15:49:52
Wash 1: ethanol removed  15:50:23
Wash 2: ethanol added  15:50:56
Wash 2: ethanol incubation finished  15:51:18
Wash 2: ethanol removed  15:51:53
Ethanol drying finished  15:56:53

Eluent added  15:59:45
Elution incubation finished  16:07:41
Beads pulled to the side  16:08:43
Eluate transferred to the new tube  16:09:11
Experiment finished  16:09:11


## Re-calibrating tips rack

In [14]:
ber.tips_rack.refill()

In [15]:
ber.calibrateRack(rack='tips')

(76.30000000000001, 149.965, 168.3)

# Protocols with Invitrogen 100bp ladder

## Diluting ladder

In [18]:
script = pd.read_csv('mixing_pattern_eppendorf.csv')

In [17]:
dna_500 = bl.createSample('eppendorf', 'dna_500', ber.samples_rack, 0, 10, 50)
dna_stock = bl.createSample('eppendorf', 'dna_stock', ber.samples_rack, 0, 9, 0)
dna_stock_2 = bl.createSample('eppendorf', 'dna_stock_2', ber.samples_rack, 0, 8, 0)
water = bl.createSample('50ml', 'water', ber.reagents_rack, 1, 1, 30000)

In [19]:
ber.pickUpNextTip()
ber.move(z=50)
ber.transferLiquid(water, dna_stock, 90)
ber.transferLiquid(water, dna_stock_2, 980)
ber.transferLiquid(dna_500, dna_stock, 10, v_immerse_dispense=1600)
ber.move(z=50)
ber.mixByScript(dna_stock, script)
ber.move(z=50)
ber.transferLiquid(dna_stock, dna_stock_2, 20, v_immerse_dispense=1600)
ber.move(z=50)
ber.mixByScript(dna_stock_2, script)
ber.move(z=50)
ber.dumpTipToWaste()

In [20]:
dna_stock_2.getVolume()

1000

## 150bp cutoff

In [21]:
ber.pickUpNextTip()
sample = bl.createSamplesToPurifyList(ber, [0])[0]
ber.transferLiquid(dna_stock_2, sample, 30, v_immerse_dispense=1600, touch_wall=False)
ber.move(z=50)
ber.dumpTipToWaste()
ber.pipetteHome()

In [22]:
oneStagePurification(ber, N_samples=1, initial_vol_list=[30], 
                     cutoff_list=[150],
                     V_avail_beads=350, V_avail_water=30000, V_avail_ethanol=3000, 
                     beads_rack=ber.samples_rack, beads_col=0, beads_row=11, 
                     sample_mix_scenario=script, beads_mix_scenario=script, 
                     T_absorb=480, T_dry=300, T_elute=480)

Experiment started  16:50:48
Beads added  16:52:55
Waiting for DNA absorption
DNA absorption finished  17:01:01
Beads pulled to the side  17:02:03
Supernatant removed  17:02:33
Wash 1: ethanol added  17:03:06
Wash 1: ethanol incubation finished  17:03:28
Wash 1: ethanol removed  17:03:59
Wash 2: ethanol added  17:04:33
Wash 2: ethanol incubation finished  17:04:54
Wash 2: ethanol removed  17:05:30
Ethanol drying finished  17:10:30

Eluent added  17:13:22
Elution incubation finished  17:21:16
Beads pulled to the side  17:22:18
Eluate transferred to the new tube  17:22:47
Experiment finished  17:22:47


## 250bp cutoff

In [23]:
ber.pickUpNextTip()
sample = bl.createSamplesToPurifyList(ber, [0])[0]
ber.transferLiquid(dna_stock_2, sample, 30, v_immerse_dispense=1600, touch_wall=False)
ber.move(z=50)
ber.dumpTipToWaste()
ber.pipetteHome()

In [24]:
oneStagePurification(ber, N_samples=1, initial_vol_list=[30], 
                     cutoff_list=[250],
                     V_avail_beads=300, V_avail_water=30000, V_avail_ethanol=3000, 
                     beads_rack=ber.samples_rack, beads_col=0, beads_row=11, 
                     sample_mix_scenario=script, beads_mix_scenario=script, 
                     T_absorb=480, T_dry=300, T_elute=480)

Experiment started  17:49:22
Beads added  17:51:08
Waiting for DNA absorption
DNA absorption finished  17:59:26
Beads pulled to the side  18:00:28
Supernatant removed  18:00:58
Wash 1: ethanol added  18:01:31
Wash 1: ethanol incubation finished  18:01:53
Wash 1: ethanol removed  18:02:24
Wash 2: ethanol added  18:02:58
Wash 2: ethanol incubation finished  18:03:19
Wash 2: ethanol removed  18:03:55
Ethanol drying finished  18:08:55

Eluent added  18:11:47
Elution incubation finished  18:19:44
Beads pulled to the side  18:20:46
Eluate transferred to the new tube  18:21:15
Experiment finished  18:21:15


 ## 450bp cutoff

In [25]:
ber.pickUpNextTip()
sample = bl.createSamplesToPurifyList(ber, [0])[0]
ber.transferLiquid(dna_stock_2, sample, 30, v_immerse_dispense=1600, touch_wall=False)
ber.move(z=50)
ber.dumpTipToWaste()
ber.pipetteHome()

In [26]:
oneStagePurification(ber, N_samples=1, initial_vol_list=[30], 
                     cutoff_list=[450],
                     V_avail_beads=250, V_avail_water=30000, V_avail_ethanol=2500, 
                     beads_rack=ber.samples_rack, beads_col=0, beads_row=11, 
                     sample_mix_scenario=script, beads_mix_scenario=script, 
                     T_absorb=480, T_dry=300, T_elute=480)

Experiment started  18:28:25
Beads added  18:30:11
Waiting for DNA absorption
DNA absorption finished  18:38:23
Beads pulled to the side  18:39:25
Supernatant removed  18:39:55
Wash 1: ethanol added  18:40:28
Wash 1: ethanol incubation finished  18:40:50
Wash 1: ethanol removed  18:41:21
Wash 2: ethanol added  18:41:55
Wash 2: ethanol incubation finished  18:42:16
Wash 2: ethanol removed  18:42:51
Ethanol drying finished  18:47:52

Eluent added  18:50:43
Elution incubation finished  18:58:38
Beads pulled to the side  18:59:40
Eluate transferred to the new tube  19:00:09
Experiment finished  19:00:09


## 1000bp cutoff

In [27]:
ber.pickUpNextTip()
sample = bl.createSamplesToPurifyList(ber, [0])[0]
ber.transferLiquid(dna_stock_2, sample, 30, v_immerse_dispense=1600, touch_wall=False)
ber.move(z=50)
ber.dumpTipToWaste()
ber.pipetteHome()

In [28]:
oneStagePurification(ber, N_samples=1, initial_vol_list=[30], 
                     cutoff_list=[1000],
                     V_avail_beads=200, V_avail_water=30000, V_avail_ethanol=2000, 
                     beads_rack=ber.samples_rack, beads_col=0, beads_row=11, 
                     sample_mix_scenario=script, beads_mix_scenario=script, 
                     T_absorb=480, T_dry=300, T_elute=480)

Experiment started  19:03:48
Beads added  19:05:47
Waiting for DNA absorption
DNA absorption finished  19:13:55
Beads pulled to the side  19:14:57
Supernatant removed  19:15:27
Wash 1: ethanol added  19:16:00
Wash 1: ethanol incubation finished  19:16:21
Wash 1: ethanol removed  19:16:53
Wash 2: ethanol added  19:17:27
Wash 2: ethanol incubation finished  19:17:48
Wash 2: ethanol removed  19:18:24
Ethanol drying finished  19:23:24

Eluent added  19:26:15
Elution incubation finished  19:34:09
Beads pulled to the side  19:35:11
Eluate transferred to the new tube  19:35:40
Experiment finished  19:35:40


## Beads volume added calculations

In [5]:
sample = bl.createSamplesToPurifyList(ber, [30])[0]

In [6]:
ber.calcBeadsVol(sample, 150)

47.34245107533334

In [10]:
ber.calcBeadsVol(sample, 1000) / 30

0.51517349496